In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import random

BASE_URL = "https://www.goodreads.com"
SHELVES = [
    "children", "fantasy", "romance", "fiction",
    "non-fiction", "history", "biography", "science",
    "humour", "memoir","classics", "mystery",
    "thriller", "young-adult", "contemporary", "self-help", "graphic-novels",
    "poetry", "crime", "philosophy"
]
books = []
seen_links = set()

headers = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/124.0 Safari/537.36"
    ),
    "Accept-Language": "en-US,en;q=0.9",
}

for shelf in SHELVES:
    print(f"\n📚 Scraping shelf: {shelf}")
    url = f"{BASE_URL}/shelf/show/{shelf}"
    resp = requests.get(url, headers=headers)
    if resp.status_code != 200:
        print(f"  ⚠️ Failed to fetch shelf '{shelf}' — status {resp.status_code}")
        continue

    soup = BeautifulSoup(resp.text, "html.parser")
    elements = soup.select("a.leftAlignedImage[title]")
    if not elements:
        print(f"  ❌ No books found for shelf '{shelf}'.")
        continue

    for a_tag in elements:
        title = a_tag["title"].strip()
        link = BASE_URL + a_tag["href"]

        # skip duplicates across shelves
        if link in seen_links:
            continue
        seen_links.add(link)

        parent_div = a_tag.find_parent("div", class_="elementList")
        author_tag = parent_div.select_one("a.authorName span") if parent_div else None
        rating_tag = parent_div.select_one("span.greyText.smallText") if parent_div else None

        author = author_tag.text.strip() if author_tag else None
        rating, pub_year = None, None

        if rating_tag:
            text = rating_tag.text.strip()
            if "avg rating" in text:
                try:
                    rating = float(text.split("avg rating")[1].split("—")[0].strip())
                except:
                    pass
            if "published" in text:
                for word in text.split():
                    if word.isdigit() and len(word) == 4:
                        pub_year = word
                        break

        books.append({
            "Shelf": shelf,
            "Title": title,
            "Author": author,
            "Rating": rating,
            "Publication Year": pub_year,
            "Link": link
        })

    print(f"  ✅ Found {len(elements)} books in shelf '{shelf}'. Total so far: {len(books)}")
    time.sleep(random.uniform(1.5, 2.5))  # polite delay between shelves

# Convert to DataFrame and save
df = pd.DataFrame(books)
df.drop_duplicates(subset=["Title", "Author"], inplace=True)
df.to_csv("goodreads_singlepage_multigenre.csv", index=False)
print(f"\n🎉 Done! Saved {len(df)} unique books → goodreads_singlepage_multigenre.csv")


📚 Scraping shelf: children
  ✅ Found 50 books in shelf 'children'. Total so far: 50

📚 Scraping shelf: fantasy
  ✅ Found 50 books in shelf 'fantasy'. Total so far: 91

📚 Scraping shelf: romance
  ✅ Found 50 books in shelf 'romance'. Total so far: 136

📚 Scraping shelf: fiction
  ✅ Found 50 books in shelf 'fiction'. Total so far: 173

📚 Scraping shelf: non-fiction
  ✅ Found 50 books in shelf 'non-fiction'. Total so far: 223

📚 Scraping shelf: history
  ✅ Found 50 books in shelf 'history'. Total so far: 261

📚 Scraping shelf: biography
  ✅ Found 50 books in shelf 'biography'. Total so far: 291

📚 Scraping shelf: science
  ✅ Found 50 books in shelf 'science'. Total so far: 329

📚 Scraping shelf: humour
  ✅ Found 50 books in shelf 'humour'. Total so far: 375

📚 Scraping shelf: memoir
  ✅ Found 50 books in shelf 'memoir'. Total so far: 398

📚 Scraping shelf: classics
  ✅ Found 50 books in shelf 'classics'. Total so far: 425

📚 Scraping shelf: mystery
  ✅ Found 50 books in shelf 'mystery'. 